In [49]:
import pandas as pd
from collections import deque, defaultdict


file_path = 'data/358_the_acc_dep_got_everything_mixed_up/'
data = pd.read_csv(file_path + 'points_task_input_2.csv')


def calculate_balances(df):
    results = []
    for user_id, user_df in df.groupby('id', sort=False):
        queue = deque()
        accrued_services = set()

        for _, row in user_df.iterrows():
            service = row['service']
            amount = row['amount']
            
            if amount > 0:
                queue.append([service, amount])
                accrued_services.add(service)
            else:
                spend = -amount
                while spend > 0 and queue:
                    serv, remaining = queue[0]
                    if remaining <= spend:
                        spend -= remaining
                        queue.popleft()
                    else:
                        queue[0][1] -= spend
                        spend = 0

        balances = defaultdict(int)
        for serv, remaining in queue:
            balances[serv] += remaining

        for serv in sorted(accrued_services):
            results.append({
                'id': user_id,
                'service': serv,
                'balance': balances[serv]
            })
    
    return pd.DataFrame(results)


balances = calculate_balances(data)
balances_sorted = balances.sort_values(by=['id', 'service']).reset_index(drop=True)
output_file_path = 'data/358_the_acc_dep_got_everything_mixed_up/calculated_balances.csv'
balances_sorted.to_csv(output_file_path, index=False)
print(f"Результаты сохранены в файл: {output_file_path}")

Результаты сохранены в файл: data/358_the_acc_dep_got_everything_mixed_up/calculated_balances_correct.csv
